In [1]:
from bs4 import BeautifulSoup
import html.parser
import json
import csv
import time
import re
import requests

urls = []
with open('Yelp.csv', 'r' ) as csv_file:
    csv_reader = csv.reader(csv_file)
    for line in csv_reader:
        url = str(line[0]).strip()
        urls.append(url)
print(len(urls))

headers = {
    'authority': 'www.yelp.com',
    'cache-control': 'max-age=0',
    'sec-ch-ua': '" Not;A Brand";v="99", "Google Chrome";v="91", "Chromium";v="91"',
    'sec-ch-ua-mobile': '?0',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.101 Safari/537.36',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'sec-fetch-site': 'none',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-user': '?1',
    'sec-fetch-dest': 'document',
    'accept-language': 'en-US,en;q=0.9,hi;q=0.8',
}

for i in range(0,len(urls)):
    url = urls[i]
    r = requests.get(url, headers=headers)
    
    status = str(r.status_code).strip()
    
    if status=="503":
        return
    soup = BeautifulSoup(r.text, 'html.parser')
    
    name,address,website,phone,amenities = [""]*5
    
    try:
        name    = str(soup.select_one('h1[class="css-11q1g5y"]').text).strip()
    except:
        pass
    

    try:
        address = str(soup.select_one('address[class]').text).strip()
    except:
        pass


    try:
        sections = soup.select('section[class*="373c0__1TRkQ"]')
        for section in sections:
            label = str(section.select_one('[class="css-16iwlls"]').text).strip()
            if label=="Amenities and More":
                amenities     = ""
                amenities_raw = section.select('[class="css-1h1j0y3"]')
                for amenity in amenities_raw:
                    amenities += str(amenity.text).strip() + ", "
                amenities = amenities.strip(", ")
    except:
        amenities = ""

    items = soup.select('section[class*="b3__373c0__q1DuY"] [class*="2__373c0__3Zpeq"]')
    for item in items:
        try:
            label  = str(item.select_one('[class="css-aml4xx"]').text).strip()
        except:
            label  = ""

        try:
            result  = str(item.select_one('[class="css-1h1j0y3"]').text).strip()
        except:
            result  = ""

        if label=="Business website":
            website = result
        if label=="phone number":
            phone = result

    f_data = [url,name,address,phone,website,amenities]
    with open(f'outputs/{str(i)}_output.csv', 'w', newline='', encoding="utf-8") as new_file:
        csv_writer = csv.writer(new_file)
        csv_writer.writerow(f_data)